# NOAA  API vignette

December 27, 2022

Vignette: NOAA API. Demonstrates how to pull weather data from NOAA using a REST API.

@author: Oscar Trevizo

## References
* NOAA API (Accessed Jan 2, 2023) 
* https://pypi.org/project/noaa-sdk/
* https://www.weather.gov/documentation/services-web-api
* https://api.weather.gov/openapi.jsonhttps://api.weather.gov/openapi.json
* https://www.weather.gov/media/documentation/docs/NWS_Geolocation.pdf
* https://github.com/paulokuong/noaa

# Import libraries

In [1]:
from noaa_sdk import noaa
import datetime
import json
import pandas as pd

# Init parameters

In [2]:
# parameters for retrieving NOAA weather data
zip_code = '60610'
country = 'US'
today = datetime.datetime.now()
past = today - datetime.timedelta(days=14)
start_date = past.strftime("%Y-%m-%dT00:00:00Z")
end_date = today.strftime("%Y-%m-%dT23:59:59Z")

# Instantiate object to connect to REST API

In [3]:
weather = noaa.NOAA()

# Pull the data

In [4]:
observations = weather.get_observations(zip_code, country, start_date, end_date)

# Accumulate data in lists

The data comes in a complex JSON. The following command pulls certain values only, for simplicity.

In [5]:
# Lists to become columns in a DataFrame
time = []
zc = []   # Optionally =, one could loop through several zip codes
wind_speed = []
temperature = []
humidity = []
wind_direction = []
pressure = []
visibility = []
description = []

observations = weather.get_observations(zip_code, country, start_date, end_date)

for obs in observations:
    time.append(obs["timestamp"])
    zc.append(zip_code)
    wind_speed.append(obs["windSpeed"]["value"])
    temperature.append(obs["temperature"]["value"])
    humidity.append(obs["relativeHumidity"]["value"])
    wind_direction.append(obs["windDirection"]["value"])
    pressure.append(obs["barometricPressure"]["value"])
    visibility.append(obs["visibility"]["value"])
    description.append(obs["textDescription"])
    

# Build time series DataFrame

In [6]:
# Build the DataFrame using dictionary
obs_df = pd.DataFrame({'time':time, 'zip_code':zc, 'temperature':temperature, 'wind_speed':wind_speed, 'humidity':humidity,
                   'wind_direction':wind_direction, 'pressure':pressure, 'pressure':pressure, 
                   'visibility':visibility, 'description':description})

obs_df.time = pd.to_datetime(obs_df['time'])
obs_df.set_index('time', inplace=True)


obs_df.head()

,zip_code,temperature,wind_speed,humidity,wind_direction,pressure,visibility,description
time,,,,,,,,
2023-01-05 17:53:00+00:00,60610,0.6,16.56,77.971736,250.0,101020.0,16090,Cloudy
2023-01-05 17:13:00+00:00,60610,NaN,NaN,NaN,NaN,NaN,11270,Cloudy
2023-01-05 16:53:00+00:00,60610,0.0,18.36,85.129292,250.0,101080.0,6440,Light Snow and Fog/Mist
2023-01-05 16:42:00+00:00,60610,0.0,22.32,85.129292,240.0,101080.0,2820,Light Snow and Fog/Mist
2023-01-05 15:53:00+00:00,60610,0.0,18.36,85.129292,260.0,101080.0,4830,Light Snow and Fog/Mist


# Forecasts

# Accumulate 12-hr data in lists

In [7]:
# Lists to become columns in a DataFrame
time = []
zc = []   # Optionally =, one could loop through several zip codes
wind_speed = []
temperature = []
wind_direction = []
description = []

forecasts = weather.get_forecasts(zip_code, country, hourly=False, type='forecast')

for fcst in forecasts:
    # print(fcst)
    time.append(fcst["startTime"])
    zc.append(zip_code)
    temperature.append(fcst['temperature'])
    wind_speed.append(fcst["windSpeed"])
    wind_direction.append(fcst["windDirection"])
    description.append(fcst["shortForecast"])    


# Build time series forecast DataFrame

In [8]:
# Build the DataFrame using dictionary
fcst_df = pd.DataFrame({'time':time, 'zip_code':zc, 'temperature':temperature, 'wind_speed':wind_speed, 
                   'wind_direction':wind_direction, 'description':description})

fcst_df.time = pd.to_datetime(fcst_df['time'])
fcst_df.set_index('time', inplace=True)


fcst_df.head()

,zip_code,temperature,wind_speed,wind_direction,description
time,,,,,
2023-01-05 12:00:00-06:00,60610,34,10 mph,WSW,Periods Of Snow Showers
2023-01-05 18:00:00-06:00,60610,31,10 mph,W,Scattered Snow Showers
2023-01-06 06:00:00-06:00,60610,34,5 to 10 mph,W,Mostly Cloudy
2023-01-06 18:00:00-06:00,60610,28,5 mph,NW,Partly Cloudy
2023-01-07 06:00:00-06:00,60610,37,5 mph,NNE,Mostly Sunny


# Accumulate hourly data in lists

In [9]:
# Lists to become columns in a DataFrame
time = []
zc = []   # Optionally =, one could loop through several zip codes
wind_speed = []
temperature = []
wind_direction = []
description = []

forecasts = weather.get_forecasts(zip_code, country, hourly=True, type='forecastHourly')

for fcst in forecasts:
    # print(fcst)
    time.append(fcst["startTime"])
    zc.append(zip_code)
    temperature.append(fcst['temperature'])
    wind_speed.append(fcst["windSpeed"])
    wind_direction.append(fcst["windDirection"])
    description.append(fcst["shortForecast"])    


# Build time series forecast DataFrame

In [10]:
# Build the DataFrame using dictionary
hrly_fcst_df = pd.DataFrame({'time':time, 'zip_code':zc, 'temperature':temperature, 'wind_speed':wind_speed, 
                   'wind_direction':wind_direction, 'description':description})

hrly_fcst_df.time = pd.to_datetime(hrly_fcst_df['time'])
hrly_fcst_df.set_index('time', inplace=True)


hrly_fcst_df.head()

,zip_code,temperature,wind_speed,wind_direction,description
time,,,,,
2023-01-05 12:00:00-06:00,60610,33,10 mph,WSW,Periods Of Snow Showers
2023-01-05 13:00:00-06:00,60610,34,10 mph,WSW,Periods Of Snow Showers
2023-01-05 14:00:00-06:00,60610,34,10 mph,WSW,Periods Of Snow Showers
2023-01-05 15:00:00-06:00,60610,34,10 mph,WSW,Periods Of Snow Showers
2023-01-05 16:00:00-06:00,60610,33,10 mph,WSW,Periods Of Snow Showers
